### Model Training

In [0]:
dbutils.fs.mv("dbfs:/FileStore/rachlenn/video", "dbfs:/mnt/lab/unrestricted/rachel.lennon@defra.gov.uk", True)

In [0]:
%pip install ultralytics

In [0]:
# Initiate YAML file
dataset_yaml = "/dbfs/mnt/lab/unrestricted/rachel.lennon@defra.gov.uk/LabelledFishYOLOv11/dataset.yaml"

yaml_content = """
path: /dbfs/mnt/lab/unrestricted/rachel.lennon@defra.gov.uk/LabelledFishYOLOv11
train: train/images
val: valid/images
nc: 1
names: ['fish']
"""

with open(dataset_yaml, "w") as f:
    f.write(yaml_content)



In [0]:
from ultralytics import YOLO
import torch

device="cuda" if torch.cuda.is_available() else "cpu"
print(device)

model = YOLO('/tmp/yolov8m.pt').to(device)

In [0]:
import os
import torch 
import torch.distributed as dist

if torch.cuda.is_available():
    dist.init_process_group(backend='nccl', init_method='env://')
torch.cuda.device(0)
 

In [0]:
os.makedirs("/tmp/runs", exist_ok=True)

In [0]:
results = model.train(
    data=dataset_yaml,
    epochs=50,
    imgsz=640,
    batch=16,
    project="/Users/rachel.lennon@defra.gov.uk/fish_yolo_experiment",
    name="fish_yolo",
    exist_ok=True,
    device=0,   
    single_cls=True
)


### Evaluation

In [0]:
# Evaluate on validation set
metrics = model.val()

print(metrics.box.map)      # mAP@0.5
print(metrics.box.map50)    # mAP@0.5
print(metrics.box.map75)    # mAP@0.75
print(metrics.box.maps)     # per-class mAP


In [0]:
results = model.predict(
    source="/dbfs/mnt/lab/unrestricted/rachel.lennon@defra.gov.uk/LabelledFishYOLOv11/valid/images",
    save=True,       # saves annotated images
    conf=0.25        # confidence threshold
)
